# 간단한 RAG(검색 지원 생성) 시스템

## 개요

이 코드는 PDF 문서를 처리하고 쿼리하기 위한 기본적인 검색 지원 생성(RAG) 시스템을 구현합니다. 시스템은 문서 콘텐츠를 벡터 저장소로 인코딩하여 관련 정보를 검색할 수 있습니다.

## 주요 구성 요소

1. PDF 처리 및 텍스트 추출
2. 관리 가능한 처리를 위한 텍스트 청크
3. FAISS 및 SDS 임베딩을 사용한 벡터 저장소 만들기
4. 처리된 문서를 쿼리하기 위한 검색기 설정
5. RAG 시스템의 평가

## 방법 세부 정보

### 문서 전처리

1. PyPDFLoader를 사용하여 PDF를 로드합니다.
2. 지정된 청크 크기와 겹치는 부분을 사용하여 재귀적 문자 텍스트 분할기를 사용하여 텍스트를 청크로 분할합니다.

### 텍스트 정리

사용자 지정 함수인 replace\_t\_with\_space를 적용하여 텍스트 청크를 정리합니다. 이는 PDF의 특정 서식 문제를 해결할 가능성이 높습니다.

### 벡터 저장소 만들기

1. SDS 임베딩을 사용하여 텍스트 청크의 벡터 표현을 만듭니다.
2. 효율적인 유사도 검색을 위해 이러한 임베딩으로 FAISS 벡터 저장소를 만듭니다.

### 검색기 설정

1. 검색기가 주어진 쿼리에 대해 가장 관련된 두 개의 청크를 가져오도록 구성됩니다.

### 인코딩 함수

`encode_pdf` 함수는 PDF를 로드, 청크화, 정리 및 벡터 저장소로 인코딩하는 전체 프로세스를 캡슐화합니다.

## 주요 기능

1. 모듈식 설계: 인코딩 프로세스는 재사용이 용이하도록 단일 함수에 캡슐화됩니다.
2. 구성 가능한 청크화: 청크 크기와 겹침을 조정할 수 있습니다.
3. 효율적인 검색: 빠른 유사도 검색을 위해 FAISS를 사용합니다.
4. 평가: RAG 시스템의 성능을 평가하는 기능이 포함되어 있습니다.

## 사용법 예제

코드에는 "기후 변화의 주요 원인은 무엇인가요?"라는 테스트 쿼리가 포함되어 있습니다. 이것은 처리된 문서에서 관련 컨텍스트를 가져오는 데 검색기를 어떻게 사용하는지 보여줍니다.

## 평가

시스템에는 `evaluate_rag` 함수가 포함되어 있어 검색기의 성능을 평가할 수 있지만 제공되는 코드에서는 특정 메트릭이 자세히 설명되어 있지 않습니다.

## 이 접근법의 이점

1. 확장성: 청크로 문서를 처리하여 큰 문서를 처리할 수 있습니다.
2. 유연성: 청크 크기 및 검색된 결과 수를 쉽게 조정할 수 있습니다.
3. 효율성: 고차원 공간에서 빠른 유사도 검색을 위해 FAISS를 활용합니다.
4. 고급 NLP와의 통합: 최첨단 텍스트 표현을 위해 SDS 임베딩을 사용합니다.

## 결론

이 단순한 RAG 시스템은 보다 복잡한 정보 검색 및 질문 응답 시스템을 구축하기 위한 견고한 기반을 제공합니다. 문서 콘텐츠를 검색 가능한 벡터 저장소에 인코딩함으로써 쿼리에 대한 응답으로 관련 정보의 효율적인 검색을 가능하게 합니다. 이 접근 방식은 특히 대형 문서 또는 문서 컬렉션 내에서 특정 정보에 빠르게 액세스해야 하는 애플리케이션에 유용합니다.

### Import libraries and environment variables

In [1]:
%pip install -qU sentence_transformers langchain_community text_generation fitz PyMuPDF faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
from helper_functions import *



### Read Docs

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### Encode document

In [4]:
from typing import (List)

import requests
from langchain_core.embeddings import Embeddings

class CustomEmbedding(Embeddings):
    def __init__(self):
        self.embed_url = 'http://sds-embed.serving.70-220-152-1.sslip.io/v1/models/embed:predict'

    def call_embed(self, url, texts):
        data = {
            "instances": texts
        }
        response = requests.post(url=url, json=data)
        result = response.json()
        return result['predictions']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        주어진 텍스트를 임베딩하여 벡터로 반환 합니다.
        """
        embed_list = self.call_embed(url=self.embed_url, texts=texts)
        return embed_list

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""

        embed_list = self.call_embed(url=self.embed_url, texts=[text])
        return embed_list[0]

In [7]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using SDS embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = CustomEmbedding()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [8]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

### Create retriever

In [9]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

### Test retriever

In [10]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Understanding Climate Change  
Chapter 1: Introduction to Climate Change  
Climate change refers to significant, long -term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past cent ury, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change.  
Historical Context  
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and  human civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which


Context 2:
driven by human activ

/home/user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
